In [1]:
import os
os.listdir(os.curdir)

In [87]:
#import mllib
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.feature import StandardScalerModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

import numpy as np
import os, csv, sys, time
from random import randint
from itertools import izip, izip_longest

In [50]:
# def get_input(fn):

fn = "cl_tiny.csv"
cur_dir = os.path.abspath(os.curdir)
input_file_path = os.path.normpath(os.path.join(cur_dir, "..", "data", fn))
print(input_file_path)

os.path.isfile(input_file_path)

C:\Users\shane\programming\cs657_mining_massive_datasets\craigslist_clustering\data\cl_tiny.csv


True

In [58]:
# [(postTitle, postingURL, postLocation, time, lat, long, address, dateRetrieved, post_date, ad), ...]
raw_ads = sc.textFile(input_file_path)

In [59]:
# set = input.take(3)

In [61]:
# [ad0, ad1, ..]
ads_rdd = raw_ads.map(lambda x: str(x.decode('utf-8', 'ignore')))
# processed_rdd = input.map(lambda x: str(x.decode('utf-8', 'ignore'))).map(lambda x: x.split(","))

In [60]:
processed_rdd.take(2)

['"Daziran Massage TherapyCall to make appointment.1314 Westgate Parkway, Suite 4 Dothan, AlabamaWe specialize in Back Walking, Deep Tissue, Sport Massage...etc$60/hour $65/hour deep tissue, we also have 90 Mins and 2 hours session.out call only within Dothan area.Open Monday to Saturday, 10am - 9pm, Sunday 1pm - 8pmshow contact infoAll major credit cards accepted    "',
 '"331 SPAWelcome to best asian massage. shiatsu and swedish, walk on your back , firm or relaxing massage.Great table shower.Our place is very clean and very comfortable.Every therapists are licensed and very professional and friendly.You\'ll be a New person,In callHalf Hr: $60.00 with table showerOne Hr: $80.00 with table showerwe do accept credit cardsopen 9 am to 10 pm , 7days a weekTel:  show contact info*from panama city or beach: one mile before to I-10 on highway 331 to your left* from Tallahassee: Left turn to highway 331 (Exit 85) , we are 1 mile from I-10 on your right side building.*I-10 from alabama(Mobil)

In [82]:
def get_tfidf(rdd):
    # While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
    # First to compute the IDF vector and second to scale the term frequencies by IDF.
    hashingTF = HashingTF()
    tf = hashingTF.transform(rdd)
    tf.cache()
    idf = IDF().fit(tf)
    tfidf = idf.transform(tf)
    # spark.mllib's IDF implementation provides an option for ignoring terms
    # which occur in less than a minimum number of documents.
    # In such cases, the IDF for these terms is set to 0.
    # This feature can be used by passing the minDocFreq value to the IDF constructor.
    idfIgnore = IDF(minDocFreq=1).fit(tf)
    tfidf_rdd = idfIgnore.transform(tf)
    # list of SparseVectors [(doc_id_i: {word_id_j: tfidfscore_j, ...}), ... }]
    return tfidf_rdd

In [84]:
tfidfIgnore.take(2)

[SparseVector(1048576, {30136: 1.3228, 60275: 2.5813, 71269: 1.7209, 115245: 2.4006, 134386: 3.872, 145380: 5.5929, 156374: 1.6308, 189356: 1.3228, 208501: 4.0871, 219495: 0.2151, 263471: 2.0577, 274465: 0.0, 282612: 1.936, 293606: 0.8604, 337582: 3.6491, 356727: 0.3429, 367721: 1.936, 411697: 1.0288, 422691: 0.6859, 430838: 0.8604, 441832: 3.2267, 463820: 1.194, 485808: 1.3718, 515947: 1.4881, 559923: 0.6008, 570917: 2.0722, 579064: 1.2907, 590058: 1.0237, 612046: 0.7259, 623040: 0.6859, 634034: 0.2985, 653179: 2.9762, 664173: 2.3662, 686161: 0.4895, 697155: 0.9491, 708149: 2.0722, 738284: 1.7209, 771266: 0.3895, 782260: 0.6008, 812399: 3.6569, 845381: 2.9037, 856375: 1.2015, 886510: 3.872, 897504: 7.744, 908498: 1.3718, 960625: 2.5813, 971619: 0.3895, 982613: 1.2015, 1004601: 0.7259, 1045730: 2.1511}),
 SparseVector(1048576, {19142: 1.1314, 30136: 0.6614, 60275: 8.3893, 71269: 3.0116, 93257: 1.6308, 104251: 2.7181, 115245: 1.7147, 134386: 5.808, 145380: 11.4009, 156374: 0.5436, 16736

In [81]:
#     sentence = "aa bb ab" * 10 + "a cd " * 10
#     localDoc = [sentence, sentence]
#     doc = sc.parallelize(localDoc).map(lambda line: line.split(" "))
#     model = Word2Vec().setVectorSize(10).setSeed(42).fit(doc)
# i think it is expecting a list of document lists [[word1, word2,...], ...]
def get_word2vec(rdd):
    word2vec = Word2Vec()
    model = word2vec.fit(ads_rdd)



In [94]:
# distributed matrix
matrix_rdd = RowMatrix(tfidf_rdd)
svd = matrix_rdd.computeSVD(2)

In [98]:
svd.V

DenseMatrix(1048576, 2, [-0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, ..., -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0], 0)